In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd 
from time import sleep, mktime
from pprint import pprint
import credenciais
from datetime import datetime, date
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--log-level=3')
url = 'https://internetbanking.caixa.gov.br/sinbc/#!nb/login'

chrome = webdriver.Chrome(
    options=options,
    service=ChromeService(ChromeDriverManager().install())
    )

In [3]:
chrome.get(url)

In [4]:
usuario = credenciais.usuario_caixa
chrome.find_element(By.ID,"nomeUsuario").send_keys(usuario)
chrome.find_element(By.NAME,"btnLogin").click()

In [5]:
# Clicar no botão 
chrome.find_element(By.ID,'lnkInitials').click()

In [6]:
# Inserção de senha no teclado virtual
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[0]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[1]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[2]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[3]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[4]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[5]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[6]}]').click()
sleep(1)
chrome.find_element(By.XPATH,f'//*[@id="teclado"]/ul/li[{credenciais.senha_caixa[7]}]').click()
sleep(1)
chrome.find_element(By.ID,'btnConfirmar').click()

In [7]:
# Caso exista o popup de promoção, clicar em fechar
try:
    chrome.find_element(By.NAME,'btnPromoFechar').click()
except Exception:
    print("Não possuía o popup de promoção")

In [8]:
# Clicando em cartões na home
chrome.find_element(
    By.XPATH,
    '//*[@id="carrosselLista"]/li[2]/div[1]/div').click()

In [9]:
# Clicando em Faturas para pegar a data de fechamento
chrome.find_element(
    By.XPATH,
    '//*[@id="submenu"]/div[3]/ul/li[2]/a').click()

In [10]:
chrome.find_element(By.XPATH,'//*[@id="linhaTabelaInicio"]/tr[2]').click()

In [11]:
elemento = chrome.find_element(By.XPATH,'//*[@id="tb_list_Inicio"]')
html_content = elemento.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find(name='table')
table_content = pd.read_html(str(table))[0]
data_fechamento = table_content[[0, 1]]
data = data_fechamento.to_dict('records')
data_definitiva = data[1][1]
print(f'A data de fechamento da próxima fatura é: {data_definitiva}')

A data de fechamento da próxima fatura é: 25/03/2023


In [12]:
# Volta para a home
chrome.find_element(
    By.XPATH,
    '//*[@id="home"]/div[1]/div[2]/div[2]/div[2]/div/div[1]/div').click()

In [13]:
# Clicando em cartões
chrome.find_element(
    By.XPATH,
    '//*[@id="carrosselLista"]/li[2]/div[1]/div').click()

In [14]:
# Histórico de compras
chrome.find_element(
    By.XPATH,
    '//*[@id="submenu"]/div[3]/ul/li[3]/a').click()

In [15]:
# Rolar página para baixo e carregar o HTML caso esteja oculto ainda
chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [16]:
elemento = chrome.find_element(By.XPATH,'//*[@id="tb_list_Inicio"]')
html_content = elemento.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find(name='table')

extrato_full = pd.read_html(str(table))[0]

extrato = extrato_full[[0, 1]]

historico_compras = {}
historico_compras['extrato'] = extrato.to_dict('records')

lista = []
caixa = []
for linha in historico_compras['extrato']:
    if 'Cancelada' in linha[0]:
        continue
    elif 'Negada' in linha[0]:
        continue
    data = linha[0].split(' - ')
    data = data[0].split('-')
    data = data[0] + '/' + data[1] + '/' + data[2]
    lista.append(data)

    description = linha[0].split('Aprovada')
    description = description[0]
    description = description[22:-1]
    lista.append(description)

    valor = linha[1].split('R$ ')
    valor = valor[1].split(' ')
    valor = 'R$ ' + valor[0]
    lista.append(valor)
    caixa.append(lista)
    lista = []
pprint(caixa)
chrome.close()

[['28/02/2023', 'DROGAFUJI', 'R$ 49,99'],
 ['28/02/2023', 'POSTO METROPOLI', 'R$ 50,00'],
 ['28/02/2023', 'DM ROKU', 'R$ 27,90'],
 ['27/02/2023', 'POSTO METROPOLI', 'R$ 50,00'],
 ['26/02/2023', 'AUTO POSTO EPTG', 'R$ 40,00'],
 ['24/02/2023', 'CACAU SHOW', 'R$ 50,70'],
 ['23/02/2023', 'SUPERCEI SUPERM', 'R$ 34,43'],
 ['23/02/2023', 'POSTO METROPOLI', 'R$ 50,00'],
 ['22/02/2023', 'CASCOL COMBUSTI', 'R$ 30,00'],
 ['22/02/2023', 'PRIMOR SUPERMER', 'R$ 37,64'],
 ['21/02/2023', 'DISTRIBUIDORA J', 'R$ 15,00'],
 ['19/02/2023', 'DISTRIBUIDORA J', 'R$ 28,00'],
 ['19/02/2023', 'SUPER ADEGA', 'R$ 120,00'],
 ['19/02/2023', 'AUTO POSTO GOLD', 'R$ 50,00'],
 ['19/02/2023', 'DROGARIA SAUDE', 'R$ 39,99'],
 ['16/02/2023', 'COSMOPOLITAN', 'R$ 9,09'],
 ['14/02/2023', 'COSMOPOLITAN', 'R$ 8,19'],
 ['12/02/2023', 'AUTO POSTO ROCH', 'R$ 50,00'],
 ['09/02/2023', 'POSTO METROPOLI', 'R$ 50,00'],
 ['09/02/2023', 'COSMOPOLITAN', 'R$ 9,00'],
 ['09/02/2023', 'TON CONSULTORIO', 'R$ 105,00'],
 ['08/02/2023', 'REDE DROG

In [17]:
dicionario_meses = {1: "janeiro", 2: "fevereiro", 3: "março",
                    4: "abril", 5: "maio", 6: "junho",
                    7: "julho", 8: "agosto", 9: "setembro",
                    10: "outubro", 11: "novembro", 12: "dezembro"}

In [18]:
fechamento_fatura = data_definitiva
aba_mes = int(fechamento_fatura.split('/')[1]) + 1
if aba_mes == 13:
    aba_mes = 1

In [19]:
# Setar o range da aba mês - Fechamento da Fatura Caixa
range_fechamento = f'{dicionario_meses[aba_mes]}!F5'

In [20]:
print(f'Fechamento da próxima fatura do Cartão Caixa: {fechamento_fatura}')

Fechamento da próxima fatura do Cartão Caixa: 25/03/2023
Histórico de compras do Cartão Caixa: [['28/02/2023', 'DROGAFUJI', 'R$ 49,99'], ['28/02/2023', 'POSTO METROPOLI', 'R$ 50,00'], ['28/02/2023', 'DM ROKU', 'R$ 27,90'], ['27/02/2023', 'POSTO METROPOLI', 'R$ 50,00'], ['26/02/2023', 'AUTO POSTO EPTG', 'R$ 40,00'], ['24/02/2023', 'CACAU SHOW', 'R$ 50,70'], ['23/02/2023', 'SUPERCEI SUPERM', 'R$ 34,43'], ['23/02/2023', 'POSTO METROPOLI', 'R$ 50,00'], ['22/02/2023', 'CASCOL COMBUSTI', 'R$ 30,00'], ['22/02/2023', 'PRIMOR SUPERMER', 'R$ 37,64'], ['21/02/2023', 'DISTRIBUIDORA J', 'R$ 15,00'], ['19/02/2023', 'DISTRIBUIDORA J', 'R$ 28,00'], ['19/02/2023', 'SUPER ADEGA', 'R$ 120,00'], ['19/02/2023', 'AUTO POSTO GOLD', 'R$ 50,00'], ['19/02/2023', 'DROGARIA SAUDE', 'R$ 39,99'], ['16/02/2023', 'COSMOPOLITAN', 'R$ 9,09'], ['14/02/2023', 'COSMOPOLITAN', 'R$ 8,19'], ['12/02/2023', 'AUTO POSTO ROCH', 'R$ 50,00'], ['09/02/2023', 'POSTO METROPOLI', 'R$ 50,00'], ['09/02/2023', 'COSMOPOLITAN', 'R$ 9,00'],

In [21]:
SERVICE_ACCOUNT_FILE = "C:\\Users\\S027668971\\Developments\\Google-sheet\\keys.json"
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [22]:
# The ID spreadsheet.
SAMPLE_SPREADSHEET_ID = '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI'
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()


In [23]:
# Inserir data de fechamento da fatura visa
sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                    range=range_fechamento,
                    valueInputOption="USER_ENTERED",
                    body={"values": [[fechamento_fatura]]}).execute()

{'spreadsheetId': '1QhNbaGgEAUzUL_FhOI5xfqTe643xI7lxmpKysub8lPI',
 'updatedRange': 'Abril!F5',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [26]:
for gasto in caixa:
    mes_gasto = int(gasto[0].split('/')[1])
    data_gasto = gasto[0]

    # Transformar data de gasto em timestamp
    timestamp_data_gasto = mktime(datetime.strptime(
                                data_gasto, '%d/%m/%Y').timetuple())
    
    # Pegar data de fechamento equivalente ao mês do gasto
    range = f'{dicionario_meses[mes_gasto+1]}!F5'
    fechamento_anterior = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range).execute().get('values', [])[0][0]
    
    # Transformar data de fechamento em timestamp
    timestamp_data_fechamento = mktime(datetime.strptime(
                                fechamento_anterior, '%d/%m/%Y').timetuple())
    
    # Condição em que determina qual mês o gasto será armazenado
    if timestamp_data_gasto < timestamp_data_fechamento:
        # Pegando todos os gastos armazenados e determinando a próxima linha vazia
        range = f'{dicionario_meses[mes_gasto+1]}!B2:D100'
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range).execute().get('values', [])
        next_empty_row = len(result) + 2

        # Condição que compara se gasto se foi armazenado
        if gasto not in result:
            # Armazenando os gastos na planilha
            range = f'{dicionario_meses[mes_gasto+1]}!B{next_empty_row}'
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range,
                        valueInputOption="USER_ENTERED",
                        body={"values": [gasto]}).execute()
    else:
        # Pegando todos os gastos armazenados e determinando a próxima linha vazia
        range = f'{dicionario_meses[mes_gasto+2]}!B2:D100'
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=range).execute().get('values', [])
        next_empty_row = len(result) + 2

        # Condição que compara se gasto se foi armazenado
        if gasto not in result:
            # Armazenando os gastos na planilha
            range = f'{dicionario_meses[mes_gasto+2]}!B{next_empty_row}'
            sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                        range=range,
                        valueInputOption="USER_ENTERED",
                        body={"values": [gasto]}).execute()
            print(data_gasto, timestamp_data_gasto, fechamento_anterior, timestamp_data_fechamento)
